# Desafio: construção de um modelo, a partir de um banco de dados fornecido pela Quantitas,  capaz de auxiliar na tomada de decisão de investimento em empresas da bolsa de valores.

## Este modelo é dividido em 4 partes:
### 1. Requisitos necessários para executar o modelo 
### 2. Extração e limpeza dos dados
### 3. Análise dos dados
### 4. Elaboração do modelo de Machine Learning


__________________________________________________________________________________________

## 1. Requisitos necessários para executar o modelo 

##### Python 3.6 ou maior

##### Bibliotecas built-in

In [1]:
# Manipulacao do banco de dados
import sqlite3 as sqlt
# Formatacao util de datas
from datetime import datetime
# Realizado de operacoes aritmeticas envolvendo datas. Subtracao de meses, por exemplo
import dateutil.relativedelta as oper_date
import io

##### Bibliotecas externas. Todas essas bibliotecas vem já inseridas na distribuiçao [Anaconda](https://www.anaconda.com/download/) do Python. É necessário elas estarem atualizadas

In [2]:
# Manipulacao rápida de vetores
import numpy as np
# Manipulação de dataframes
import pandas as pd
# Extração de dados diretamente da web
import requests as req
# Elaboração de gráficos
import seaborn as sns
# Elaboração de gráficos
import matplotlib.pyplot as plt
# Elaboração do modelo de Machine Learning
from sklearn.linear_model import LinearRegression
# Teste da eficácia do modelo de Machine Learning
from sklearn import metrics
# Setagem de algumas configurações necessárias 
%matplotlib inline
sns.set(style="whitegrid")

### 2. Extração e limpeza dos dados

#### Os dados fornecidos estão no formato banco de dados relacional, logo são necessários alguns comandos em SQL para extrair os dados necessários para o modelo.

##### Querie pra extração das tuplas cujo atributo "Cod. Mun" seja igual ao código IBGE das cidades onde estão presentes Alpargatas/Grendene e que o atributo "NCM" começe por 6402, que é o NCM para "Outros calçados com sola exterior e parte superior de borracha ou plásticos"

In [3]:
querie_seleciona = """select * from MDIC_Exports where "Cod. Mun" in {} and 
"NCM" in (64029990, 64029900, 64029100, 64022000, 64029190, 64021900)"""
grendene_cidades = "(4307906, 4304804,  2312908, 2931350, 2304202, 2304400)"
alpargatas_cidades = "(2604007, 2504009, 3143302, 3530607)"

##### Execução dos queries acima descritos. Os resultados das consultas "querie_seleciona" são enviadas para 2 objetos da classe pandas.DataFrame, um para cada empresa. Essa classe facilita bastante a manipulação dos dados

In [ ]:
con = sqlt.connect("big_data2.db")
cursor = con.cursor()
grendene_db = pd.read_sql_query(querie_seleciona.format(grendene_cidades), con)
alpargatas_db = pd.read_sql_query(querie_seleciona.format(alpargatas_cidades), con)
con.close()

##### Amostra dos Dataframes

In [ ]:
grendene_db.head(3)

In [ ]:
alpargatas_db.head(3)

##### Informações sobre os DataFrames

In [ ]:
grendene_db.info()
print("--------------------------")
alpargatas_db.info()

##### Alguns atributos são desnecessários, estão com nomes ruins para manipulação ou estão em um formato não adequado para a análise, A  função "data_clean_db"  corrige isso.

In [ ]:
def data_clean_db(dataframe):
    
    # Remoção de atributos desnecessários
    dataframe.drop(axis=1,columns=['Cod. Mun',"Desc. Mun", 'NCM', "Desc. NCM"],inplace=True)
    
    # Renomeação dos atributos
    dict_nomes_colunas = {"month":"data","US$ de P1":"financeiro",
                          "Kg Liq de P1":"peso","Qtd de P1":"quantidade"}
    dataframe.rename(columns=dict_nomes_colunas,inplace=True)
    
    # Alteração de formatos. O atributo "data" é convertido para o tipo datetime e os atributos "financeiro" e "peso" para o tipo int
    dataframe.data = dataframe.data.apply(
        lambda data: datetime.strptime(data.split()[0], "%Y-%m-%d"))
    dataframe.set_index("index", inplace=True)
    dataframe.financeiro = dataframe.financeiro.astype("int")
    dataframe.peso = dataframe.peso.astype("int") 
    
    return dataframe

In [ ]:
grendene_db = data_clean_db(grendene_db)
alpargatas_db = data_clean_db(alpargatas_db)

In [ ]:
grendene_db.head(3)

In [ ]:
alpargatas_db.head(3)

In [ ]:
grendene_db.info()
print("--------------------------")
alpargatas_db.info()

##### Os dataframes estao organizados de forma mensal,  porém os dados financeiro são fornecidos pelas empresas trimestralmente. Logo é necessário  agrupar os dados dos dataframes em trimestres. A função "get_meses_por_trimestres" realiza isso
######  - Trimestre 1: Jan | Fev | Mar
######  - Trimestre 2: Abr | Mai | Jun
######  - Trimestre 3: Jul  | Ago | Set
######  - Trimestre 4: Out | Nov | Dez

##### O dataframe retornado dessa função terá os seguintes atributos
###### - 1M_quantidade: Quantidade de itens exportados no primeiro mês do trimestre. Formato int 
###### - 2M_quantidade: Quantidade de itens exportados no segundo mês do trimestre.  Formato int 
###### - 3M_quantidade: Quantidade de itens exportados no terceiro mês do trimestre. Formato int 
###### - acumulado_quantidade: A soma dos 3 atributos acima listados. Formato int 
###### - acumulado_peso:  Total exportado em Kg no trimestre. Formato int 
###### - acumulado_financeiro:  Total exportado em Dólar no trimestre.. Formato int 
###### - trimestre: ano/mês final/01. Formato datetime


In [ ]:

def get_meses_por_trimestres(db):
    
    # Lista doos ano de 2003 até 2017
    anos = list(range(2003,2018))
    # Lista dos meses finais de cada trimestre
    trimestres = [3, 6, 9, 12]
    
    # Instanciação do dataframe que será retornado
    balanco_mensal = pd.DataFrame(columns=
            ["1M_quantidade","2M_quantidade","3M_quantidade",
             "acumulado_quantidade","acumulado_peso","acumulado_financeiro", "trimestre"])
    
    # Em alguns trimestres não há exportaçao no mês final. 
    mes3 = (lambda exportacao: exportacao[2] if len(exportacao) == 3 else 0)
    
    for ano in anos:
        for trimestre in trimestres:
            trimestre_ = datetime(ano, trimestre, 1)
            # Seleciona do dataframe todos as linhas cujo data seja anterior ou igual a "trimestre_" e posterior a ("trimestre_" - 3 meses)
            tabela_trimestral = db[(db.data <= trimestre_) & 
                      (db.data > trimestre_ - oper_date.relativedelta(months=3))]
            
            # Obtém a quantidade de exportação de cada mes do trimestre
            quantidade_trimestral = tabela_trimestral.groupby("data").quantidade.sum()
            
            # Inserção dos dados na tabela a ser retornada
            balanco_mensal = balanco_mensal.append(
                    {"1M_quantidade": quantidade_trimestral[0],
                    "2M_quantidade": quantidade_trimestral[1],
                     "3M_quantidade": mes3(quantidade_trimestral),
                     "acumulado_quantidade": quantidade_trimestral.sum(),
                     "acumulado_peso": tabela_trimestral.peso.sum(),
                    "acumulado_financeiro": tabela_trimestral.financeiro.sum(),
                     "trimestre": trimestre_
                    }, ignore_index=True)
            
    balanco_mensal.set_index("trimestre",inplace=True)
    
    # Seta como int o tipo de todos os atributos exceto o atributo "trimestre"
    balanco_mensal.iloc[:,0:7] = balanco_mensal.iloc[:,0:7].astype("int")
    
    return balanco_mensal

In [ ]:
balanco_mensal_gren = get_meses_por_trimestres(grendene_db)
balanco_mensal_alp = get_meses_por_trimestres(alpargatas_db)

In [ ]:
balanco_mensal_alp.head(3)

In [ ]:
balanco_mensal_gren.head(3)

##### Dados trimestrais Alpargatas
###### Os dados trimestrais de exportação da Alpargatas que eu encontrei  estão no formato pdf, sendo que para cada trimestre esse pdf está com uma organização diferente. Isso dificulta bastante a extração automática dos dados. Por isso, para a Alpargatas optei por utilizar os dados fornecidos pela Quantitas no arquivo xlsx

In [ ]:
# Abertura do arquivo 
balanco_trimestral_alp = pd.read_excel("Exportação calçados.xlsx")

# Extração das colunas referentes a Alpargatas
balanco_trimestral_alp = balanco_trimestral_alp[
    ["Unnamed: 2","Volume exportado Alpargatas"]]

# Renomeação dos atributos
balanco_trimestral_alp.rename(columns={
    "Unnamed: 2":"trimestre","Volume exportado Alpargatas":"total_exportado"},inplace=True)

# Os trimestre já estão no formato datatime, porém o atributo "horas" está presente. É precisor eliminar o atributo "horas"
balanco_trimestral_alp.trimestre = balanco_trimestral_alp.trimestre.apply(
    lambda data:data.date())

# Eliminação das linhas com valores NaN
balanco_trimestral_alp.dropna(inplace=True)

# Os trimestre já estão no formato datatime, porém o atributo "horas" est. É precisor eliminar o atributo "horas"
# Setagem do atributo "total_exportado" como tipo int
balanco_trimestral_alp.total_exportado = balanco_trimestral_alp.total_exportado.astype("int")

# Setagem do atributo "trimestre" como index do dataframe
balanco_trimestral_alp.reset_index(inplace=True)  
balanco_trimestral_alp.drop(columns="index",inplace=True)
balanco_trimestral_alp.set_index("trimestre",inplace=True)
balanco_trimestral_alp.head(4)

##### Dados trimestrais da Grendene

In [ ]:
# Abertura do arquivo e leitura do sheet "Volumes"
url = "http://ri.grendene.com.br/Arquivos/Grendene_Informacoes_Financeiras_Download.xls"
balanco_trimestral_gren = pd.read_excel(url, encoding="latin-1",sheet_name="Volumes")

# Extraçao das linhas 2 e 4. Essas linha serão respectivamente as colunas "trimestre" e "total_exportado" do dataframe "balanco_trimestral_gren"
balanco_trimestral_gren.drop(columns="Grendene S.A.",inplace=True)
balanco_trimestral_gren = pd.DataFrame({"trimestre":balanco_trimestral_gren.iloc[2],
                                  "total_exportado":balanco_trimestral_gren.iloc[4]})

# Resetagem do index
balanco_trimestral_gren.reset_index(inplace=True)
balanco_trimestral_gren.drop(axis=1,columns=["index"],inplace=True)

# É necessario tirar as linhas que não estam com o atributo "trimestre" no formato trimestre+T+ano  
balanco_trimestral_gren = balanco_trimestral_gren[
                        balanco_trimestral_gren.trimestre.str[1] == "T"]

# Setagem do atributo "trimestre" como formato datetime
def get_datetime(string):
    mes, ano = string.split("T")
    ano = "20" + ano
    dict_mes = {"1":3, "2":6, "3":9, "4":12}
    return datetime(int(ano), dict_mes[mes], 1)

balanco_trimestral_gren.trimestre = balanco_trimestral_gren.trimestre.apply(
    lambda string:get_datetime(string))

# Correção do ordem de grandeza do atributo "total_exportado"
balanco_trimestral_gren.total_exportado = balanco_trimestral_gren.total_exportado.apply(
    lambda exportacao:exportacao*1000)

# Setagem do atributo "trimestre" como index
balanco_trimestral_gren.set_index("trimestre",inplace=True)
balanco_trimestral_gren.head(4)

##### Os dataframe mensais e trimestrais de cada empresa estão com shapes diferentes. É preciso corigir isso

In [ ]:
balanco_mensal_gren = balanco_mensal_gren.iloc[16:]
balanco_trimestral_gren = balanco_trimestral_gren.iloc[:-1,]

balanco_mensal_gren.info()
print("###########################################")
balanco_trimestral_gren.info()

In [ ]:
balanco_mensal_alp = balanco_mensal_alp.iloc[28:]
balanco_mensal_alp.info()
print("##########################")
balanco_trimestral_alp.info()

##### União dos dataframes mensais e trimestrais de cada empresa. Esse formato facilita a análise

In [ ]:
relacao_mensal_trimestral_gren = balanco_mensal_gren.copy()
relacao_mensal_trimestral_gren["quantidade_anunciada_trimestre"] = balanco_trimestral_gren

relacao_mensal_trimestral_alp = balanco_mensal_alp.copy()
relacao_mensal_trimestral_alp["quantidade_anunciada_trimestre"] = balanco_trimestral_alp

_____________________________

## 3. Análise dos dados

#### Alpargatas: Podemos ver que não há um regressão linear muito forte entre o atributo "quantidade_anunciada_trimestre" e os demais atributos

In [ ]:
sns.pairplot(relacao_mensal_trimestral_alp)

#### Grendene: Podemos ver que há um regressão linear forte entre o atributo "quantidade_anunciada_trimestre" e os demais atributos. Nos testes de eficiência do método de machine learning que será abordado em seguida foi verificado que utilizar os atributos "1M_quantidade",  "2M_quantidade" e "3M_quantidade" são a melhor combinação para prever  "quantidade_anunciada_trimestre" 

In [ ]:
sns.pairplot(relacao_mensal_trimestral_gren)

_____________________________________________________________

## 4. Elaboração do modelo de Machine Learning


### - Alpargatas

##### Através da análise do pairplot de relacao_mensal_trimestral_alp foi verificado que não há uma regressão linear evidente entre "quantidade_anunciada_trimestre", porém é valido ao mesmo testar a eficiência de um método de regressão linear com seus dados. Os atributos que representarão os coeficientes dessa equação linaear serão "1M_quantidade",  "2M_quantidade" e "3M_quantidade"

In [ ]:
# Instanciação de um objeto da classe LinearRegression
lm_alp = LinearRegression()

#Separação dos dados entre entrada/saida e treino/teste
x_train_alp = balanco_mensal_alp.iloc[:22,:3]
x_test_alp = balanco_mensal_alp.iloc[22:,:3]
y_train_alp = balanco_trimestral_alp.iloc[:22]
y_test_alp = balanco_trimestral_alp.iloc[22:]

# Treinamento do objeto LinearRegression
lm_alp.fit(x_train_alp, y_train_alp)

# Impressão dos coeficiente da regressão
coeff_df_alp = pd.DataFrame(lm_alp.coef_[0],x_train_alp.columns,columns=['Coefficient'])
coeff_df_alp

##### O conjunto resultante da subtração da quantidade de exportaçao real pela quantidade de exportação prevista possui curva de distribuição normal, o que é um bom sinal.

In [ ]:
predictions_alp = lm_alp.predict(x_test_alp)
sns.distplot(y_test_alp-predictions_alp)

##### Gráfico EXPORTAÇÕES REALIZADAS x EXPORTAÇÕES PREVISTAS: Os valores são bem diferentes, porém é perceptível, principalmente entre 2015/8 e 2017/2, que a linha "exportações realizadas" é parecida com a linha "exportações previstas" transladada 1 trimestre para a direita. Não consegui identificar o que causa isso. A extraçao e limpeza dos dados foram realizadas da forma correta. Isso ocorra talvez por causa de alguma regra de negócio da Alpagartas, algo como exportar somente aquilo que foi produzido 1 trimestre antes

In [ ]:
a = y_test_alp.index
c = y_test_alp
d = predictions_alp
fig, ax = plt.subplots(figsize=(20,8))
ax.plot(a, c, 'red', label='Exportações realizadas')
ax.plot(a, d, 'green', label='Exportações previstas')
legend = ax.legend(loc='upper center', shadow=True, fontsize='x-large')
plt.ylabel("Quantidade exportada (em milhões de unidades)")
plt.xlabel("Trimestre")
plt.xticks(rotation='vertical')
plt.show()


##### Métricas de avaliação

In [ ]:
print('MAE:', metrics.mean_absolute_error(y_test_alp, predictions_alp))
print('MSE:', metrics.mean_squared_error(y_test_alp, predictions_alp))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test_alp, predictions_alp)))

### - Grendene

In [ ]:
# Instanciação de um objeto da classe LinearRegression
lm_gren = LinearRegression()

#Separação dos dados entre entrada/saida e treino/teste
x_train_gren = balanco_mensal_gren.iloc[:30,:3]
x_test_gren = balanco_mensal_gren.iloc[30:,:3]
y_train_gren = balanco_trimestral_gren.iloc[:30]
y_test_gren = balanco_trimestral_gren.iloc[30:]


# Treinamento do objeto LinearRegression
lm_gren.fit(x_train_gren, y_train_gren)

# Impressão dos coeficientes da regressão
coeff_df_gren = pd.DataFrame(lm_gren.coef_[0],x_train_gren.columns,columns=['Coefficient'])
coeff_df_gren

##### Histograma residual

In [ ]:
plt.figure(figsize=(6,3))
predictions_gren = lm_gren.predict(x_test_gren)
sns.distplot(y_test_gren-predictions_gren)

##### Gráfico EXPORTAÇÕES REALIZADAS x EXPORTAÇÕES PREVISTAS: valores parecidos exceto em 2014/12. Uma explicação para isso é que nesse trimestre e em 2013/12 houve regressão linear inversa do que foi produzido mensalmente sobre o que foi exportado no final do trimestre, diferente de outros trimestres onde houve regressão linear direta

In [ ]:
a = y_test_gren.index
c = y_test_gren
d = predictions_gren
fig, ax = plt.subplots(figsize=(20,5))
ax.plot(a, c, 'red', label='Exportações realizadas')
ax.plot(a, d, 'green', label='Exportações previstas')
legend = ax.legend(loc='upper center', shadow=True, fontsize='x-large')
plt.ylabel("Quantidade exportada (em milhões de unidades)")
plt.xlabel("Trimestre")
plt.xticks(rotation='vertical')
plt.show()

##### Métricas de avaliação

In [ ]:
print('MAE:', metrics.mean_absolute_error(y_test_gren, predictions_gren))
print('MSE:', metrics.mean_squared_error(y_test_gren, predictions_gren))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test_gren, predictions_gren)))